In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "https://www.etsmtl.ca/Etudes/Premier-cycle/Baccalaureat-genie-mecanique#Cours-et-concentrations"

In [5]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
#chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [10]:
driver.get(url)

# 1. Collect course link texts for webdriver to click on

In [8]:
driver.find_element_by_id("hs-eu-confirmation-button").click()

In [11]:
driver.find_element_by_class_name("leadinModal-close").click()

In [12]:
page_soup = soup(driver.page_source, 'lxml')

In [22]:
containers = page_soup.find("div", {"id": "panneau-7", "data-tab": "tab-0952270e-5604-4987-a26e-430c8e7b781e"}).find("table", {"class": "table-cours"}).findAll("a")
len(containers)

29

In [24]:
link_texts = [container.text.strip() for container in containers]
link_texts

['Chimie et matériaux',
 'Méthodes de communication en génie mécanique',
 'Gestion et économie des projets d’ingénierie',
 'Introduction à la programmation en génie mécanique',
 'Calcul différentiel et intégral',
 'Algèbre linéaire et analyse vectorielle',
 'Équations différentielles',
 'Probabilités et statistiques',
 "Statique de l'ingénieur",
 'Développement de produits assisté par ordinateur',
 'Technologie des matériaux',
 'Dynamique',
 'Thermodynamique',
 'Technologie de fabrication',
 'Résistance des matériaux',
 'Mécanique des fluides',
 'Méthode des éléments finis des corps déformables',
 'Conception vibratoire et dynamique des structures',
 'Éléments de machines',
 'Transfert de chaleur',
 'Circuits électriques et électrotechnique',
 'Projet de conception de machines',
 'Mécatronique',
 "Projet de fin d'études en génie mécanique",
 'Encadrement de la profession et éthique professionnelle',
 'Électricité et magnétisme',
 'Développement professionnel et initiation à la santé et

# 2. Test run - try to scrape the first course

In [26]:
driver.find_element_by_link_text("Cours obligatoires pour tous les étudiants").click()

In [27]:
driver.find_element_by_link_text(link_texts[0]).click()

In [28]:
page_soup = soup(driver.page_source, 'lxml')

In [29]:
course_title = page_soup.find("span", {"class": "fiche__header"}).text.strip()
course_title

'CHM131\n\n                \nChimie et matériaux'

In [30]:
course_code = re.split("\s+", course_title)[0]
course_code

'CHM131'

In [32]:
course_name = " ".join(re.split("\s+", course_title)[1:])
course_name

'Chimie et matériaux'

In [34]:
course_desc = page_soup.find("div", {"class": "fiche__descr"}).text.strip().replace("\n", " ")
course_desc

'Faire le lien entre la structure atomique de la matière et son comportement observable. Les objectifs propres à ce cours sont la compréhension de la structure de la matière, des lois régissant les comportements des solides, des liquides et des gaz et les mécanismes de réactions chimiques. Les notions théoriques sont complétées par la présentation de procédés industriels ou de phénomènes naturels. Grandeurs physiques et systèmes d’unités. Notion de procédé, bilan de matière. Structure de la matière : atomes, liaisons chimiques, molécules. Tableau périodique. Caractérisation des mélanges. Gaz parfaits : masse volumique, pression partielle, masse molaire des mélanges, équilibre liquide-vapeur; humidité de l’air et procédés d’humidification, déshumidification et séchage. Réactions chimiques : stœchiométrie et thermochimie, efficacité des procédés, combustion. Liquides : liaisons intermoléculaires. Solutions liquides : équilibre liquide-vapeur, solubilité des sels, acides et bases, solubil

In [35]:
driver.back()

# 3. Test run successful! Implement automation script to scrape all courses

In [40]:
#test run complete. Automation loop:
from selenium.common.exceptions import NoSuchElementException

course_codes = []
course_names = []
course_descs = []
counter = 0
    
for link_text in link_texts:
    #go to course page
    try:
        link = driver.find_element_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue

    time.sleep(1)
    link.click()    
    time.sleep(3)
    
    #scrape course info
    page_soup = soup(driver.page_source, 'lxml')
    course_title = page_soup.find("span", {"class": "fiche__header"}).text.strip()
    course_code = re.split("\s+", course_title)[0]
    course_name = " ".join(re.split("\s+", course_title)[1:])
    course_desc = page_soup.find("div", {"class": "fiche__descr"}).text.strip().replace("\n", " ").replace("\xa0", " ").replace("\t", " ")
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    print("Scraped ", course_codes[-1], course_names[-1], "\n", course_descs[-1])
    counter += 1    
    print("{} courses scraped".format(counter))
    print()
    
    #go to course list page
    driver.back()
    time.sleep(3)
        
print("Finished scraping {} courses".format(counter))


Scraped  CHM131 Chimie et matériaux 
 Faire le lien entre la structure atomique de la matière et son comportement observable. Les objectifs propres à ce cours sont la compréhension de la structure de la matière, des lois régissant les comportements des solides, des liquides et des gaz et les mécanismes de réactions chimiques. Les notions théoriques sont complétées par la présentation de procédés industriels ou de phénomènes naturels. Grandeurs physiques et systèmes d’unités. Notion de procédé, bilan de matière. Structure de la matière : atomes, liaisons chimiques, molécules. Tableau périodique. Caractérisation des mélanges. Gaz parfaits : masse volumique, pression partielle, masse molaire des mélanges, équilibre liquide-vapeur; humidité de l’air et procédés d’humidification, déshumidification et séchage. Réactions chimiques : stœchiométrie et thermochimie, efficacité des procédés, combustion. Liquides : liaisons intermoléculaires. Solutions liquides : équilibre liquide-vapeur, solubili

Scraped  MAT165 Algèbre linéaire et analyse vectorielle 
 Au terme de ce cours, l’étudiant comprendra et maîtrisera les notions fondamentales d'algèbre matricielle et d'analyse vectorielle. Vecteurs, algèbre et géométrie vectorielle, produits scalaires, vectoriels et mixtes, fonctions vectorielles à une variable et applications. Transformations linéaires, matrices, déterminants, inversion de matrices, systèmes d'équations linéaires, valeurs propres et vecteurs propres. Fonctions à plusieurs variables, dérivées partielles, dérivées directionnelles, gradient; applications géométriques : courbes de niveaux, optimisation, plans tangents. Intégrales doubles et triples; applications : calcul de surfaces, volumes, centres de gravité, moments d'inertie. Champ vectoriel, divergence et rotationnel, intégrales de lignes et de surfaces; théorèmes de Green, Stokes et de la divergence. Séances de travaux pratiques composées d'exercices choisis pour illustrer et compléter la théorie vue en classe.
6 

Scraped  MEC222 Dynamique 
 Au terme de ce cours, l’étudiant aura acquis les concepts fondamentaux de la dynamique des corps solides et sera en mesure de les appliquer à l'étude dynamique des mécanismes et des machines. À la fin de ce cours, l’étudiant sera en mesure :  d’appliquer les concepts et principes fondamentaux de la dynamique dans l’étude des mécanismes élémentaires; d’établir des équations du mouvement des particules et des corps rigides par la méthode de forces et d’accélérations et par les méthodes de l’énergie et de la quantité de mouvement; de calculer les efforts nécessaires pour produire un mouvement et déterminer le mouvement selon les chargements dynamiques appliqués; de modéliser et simuler le mouvement des systèmes mécaniques articulés à l’aide d’un logiciel multi-corps.  Cinématique des particules et des corps rigides: position, vitesse et accélération instantanées, déplacements, mouvements rectiligne et curviligne, repère en translation, mouvement relatif, systèm

Scraped  MEC423 Méthode des éléments finis des corps déformables 
 Ce cours vise à initier les étudiants à la méthode des éléments finis en traitant des problèmes de déformation structurale et de conduction thermique. À la fin de ce cours, l’étudiant sera en mesure :  d’appliquer les mathématiques de base de la méthode des éléments finis à des modèles simples; de résoudre des problèmes d’éléments finis en programmant la solution des équations d’équilibre; de modéliser des problèmes simples et complexes en utilisant un logiciel commercial de calcul par éléments finis et interpréter les résultats; de concevoir et analyser des corps déformables soumis à des conditions frontières thermiques ou structurales; de rédiger un rapport de qualité professionnelle présentant toutes les étapes d’une conception fondée sur une analyse par éléments finis.  Mathématiques matricielles et formulations intégrales. Fonctions d’interpolation, degrés de liberté, assemblage de matrices. Éléments finis de type 

Scraped  MEC788 Mécatronique 
 À la fin de ce cours, l’étudiant sera en mesure :  d’appliquer différentes techniques de modélisation et de simulation pour décrire des systèmes dynamiques; d’analyser les performances et la stabilité d'un système dynamique; de concevoir et ajuster les paramètres de systèmes de commande continue PID;  de sélectionner et d’intégrer les composantes d'un système de commande (capteurs, système de commande et actionneurs) de façon à pouvoir commander efficacement un procédé industriel.  Concept de systèmes et de composantes. Formulation des modèles mathématiques de systèmes physiques, transformée de Laplace, schéma-bloc, fonctions de transfert. Réponse des systèmes du premier et du deuxième ordre. Analyse dans les domaines temporels et fréquentiels. Application aux systèmes mécaniques, électriques, hydrauliques et thermiques. Terminologie et concepts de base de la commande automatique : régulateur, suiveur, systèmes en boucle ouverte et en boucle fermée, servo

Finished scraping 29 courses


# 4. Inspect and write to CSV

In [41]:
course_codes

['CHM131',
 'COM129',
 'GIA410',
 'INF135',
 'MAT145',
 'MAT165',
 'MAT265',
 'MAT350',
 'MEC111',
 'MEC129',
 'MEC200',
 'MEC222',
 'MEC240',
 'MEC300',
 'MEC329',
 'MEC335',
 'MEC423',
 'MEC525',
 'MEC528',
 'MEC532',
 'MEC546',
 'MEC592',
 'MEC788',
 'MEC795',
 'PEP110',
 'PHY332',
 'PRE011',
 'TIN503',
 'TIN504']

In [42]:
course_names

['Chimie et matériaux',
 'Méthodes de communication en génie mécanique',
 'Gestion et économie des projets d’ingénierie',
 'Introduction à la programmation en génie mécanique',
 'Calcul différentiel et intégral',
 'Algèbre linéaire et analyse vectorielle',
 'Équations différentielles',
 'Probabilités et statistiques',
 "Statique de l'ingénieur",
 'Développement de produits assisté par ordinateur',
 'Technologie des matériaux',
 'Dynamique',
 'Thermodynamique',
 'Technologie de fabrication',
 'Résistance des matériaux',
 'Mécanique des fluides',
 'Méthode des éléments finis des corps déformables',
 'Conception vibratoire et dynamique des structures',
 'Éléments de machines',
 'Transfert de chaleur',
 'Circuits électriques et électrotechnique',
 'Projet de conception de machines',
 'Mécatronique',
 "Projet de fin d'études en génie mécanique",
 'Encadrement de la profession et éthique professionnelle',
 'Électricité et magnétisme',
 'Développement professionnel et initiation à la santé et

In [43]:
course_descs

['Faire le lien entre la structure atomique de la matière et son comportement observable. Les objectifs propres à ce cours sont la compréhension de la structure de la matière, des lois régissant les comportements des solides, des liquides et des gaz et les mécanismes de réactions chimiques. Les notions théoriques sont complétées par la présentation de procédés industriels ou de phénomènes naturels. Grandeurs physiques et systèmes d’unités. Notion de procédé, bilan de matière. Structure de la matière : atomes, liaisons chimiques, molécules. Tableau périodique. Caractérisation des mélanges. Gaz parfaits : masse volumique, pression partielle, masse molaire des mélanges, équilibre liquide-vapeur; humidité de l’air et procédés d’humidification, déshumidification et séchage. Réactions chimiques : stœchiométrie et thermochimie, efficacité des procédés, combustion. Liquides : liaisons intermoléculaires. Solutions liquides : équilibre liquide-vapeur, solubilité des sels, acides et bases, solubi

In [44]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs
})

df

,Course Number,Course Name,Course Description
0,CHM131,Chimie et matériaux,Faire le lien entre la structure atomique de l...
1,COM129,Méthodes de communication en génie mécanique,Acquérir une plus grande maîtrise des divers o...
2,GIA410,Gestion et économie des projets d’ingénierie,Développer une capacité à intégrer de façon ap...
3,INF135,Introduction à la programmation en génie mécan...,Cours destiné aux étudiants n’ayant jamais pro...
4,MAT145,Calcul différentiel et intégral,"Au terme de ce cours, l’étudiant maîtrisera de..."
5,MAT165,Algèbre linéaire et analyse vectorielle,"Au terme de ce cours, l’étudiant comprendra et..."
6,MAT265,Équations différentielles,"Au terme de ce cours, l’étudiant aura acquis d..."
7,MAT350,Probabilités et statistiques,"Au terme de ce cours, l’étudiant sera initié a..."
8,MEC111,Statique de l'ingénieur,Maîtriser les concepts fondamentaux de la stat...
9,MEC129,Développement de produits assisté par ordinateur,"À la fin du cours, l’étudiant sera en mesure :..."


In [45]:
df.to_csv('ETS_GenieMecanique_Core_Courses.csv', index = False)

In [46]:
driver.quit()